In [164]:
import json
from functools import partial
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re


def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def test(origin_mapping,idx):
    print(len(idx))
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    checker(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    vv = set(values)
    for i in range(len(keys)):
        if keys[i] in values:
            nm[keys[i]] = keys[i]
        #nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    checker(nm,origin_mapping)

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = []
    for i in range(len(k)):
        if i != idx[i]:      
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                temp = []
                flag = False
    if len(temp) != 0:
        mark += temp
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res

def classify(origin_mapping):
    same, prefix, infix, postfix, others = [], [], [], [], []
    for k,v in origin_mapping.items():
        index = v.find(k)
        if k == v:
            same += [k]
        elif index != -1:
            if index == 0:
                prefix += [k]
            elif v[index:] == k:
                postfix += [k]
            else:
                infix += [k]
        else:
            others += [k]
    print('same: {}\nprefix: {}\ninfix: {}\npostfix: {}\nothers: {}'.format(len(same),len(prefix),len(infix),len(postfix),len(others)))
    return same, prefix, infix, postfix, others

def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

def mycmp(rhs,lhs):
    if rhs[3] == lhs[3]:
        for i in range(len(rhs[2])):
            if int(rhs[2][i]) > int(lhs[2][i]):
                return 1
            elif int(rhs[2][i]) == int(lhs[2][i]):
                continue
            else:
                return -1
        return 1
    elif rhs[3] > lhs[3]:
        return 1
    else:
        return -1
def mycmp1(rhs,lhs):
    if rhs[3] == lhs[3]:
        for i in range(len(rhs[2])):
            if int(rhs[2][i]) > int(lhs[2][i]):
                return 1
            elif int(rhs[2][i]) == int(lhs[2][i]):
                continue
            else:
                return -1
        return 1
    elif rhs[3] > lhs[3]:
        return 1
    else:
        return -1

if __name__ == '__main__':
    case = 1
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    nf = '../input/case{}_in.json'.format(case)
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()
    # rule based classifying
    same, prefix, infix, postfix, others = classify(origin_mapping)
   
    # only remove same
    same_set = set(same)
    exclusive_same = dict([item for item in origin_mapping.items() if item[0] not in same_set])
 
    mmm = sorted(list(exclusive_same.items()),key=lambda x: x[0])

    k,v = zip(*mmm)
    k = list(k)
    v = list(v)          
    test = list(zip(k,v,[ i for i in range(len(k))]))

    extract = []
    aaaa = []
    for i in range(len(test)):
        integer = re.findall('([0-9]+)',test[i][0])
        if len(integer) != 0:
            aa = list(re.finditer('([0-9]+)',test[i][0]))
            temp = test[i][0][:]
            for j in range(len(aa)):
                temp = temp.replace(aa[j].group(0),'',1)
            aaaa.append(temp)
        else:
            aaaa.append(test[i][0])
        extract.append(integer)
    
    
    t1,t2,t3 = zip(*test)
    tttt = list(zip(t1,t2,extract,aaaa))
    
    #tttt = sorted(tttt,key=lambda x:x[3])
    tttt = sorted(tttt,key=cmp_to_key(mycmp))
    
    for i in range(200):
        print(tttt[i])
        
    f1,f2,f3,f4 = zip(*tttt)
    fff = sorted()
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    
    
    v, idx = zip(*mm) 
    
    idx = list(idx)
    v = list(v)
    mark = consecu_int_run_length(idx)
    print('len of str of mark: {}'.format(len(str(mark))))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('len of str of sporadic_seg: {}'.format(len(str(sporadic_seg))))
    document1 = """import json as a,sys 
w={}
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
k,v=set(k),set(v)
r=k&v
c=list(r)
m=dict(zip(c,c))
k,v=s(k-r),s(v-r)
l=len
g=range
for i in g(l(k)):
    m[k[i]]=v[i]
for i in g(0,l(w),3):
    if w[i+2]!=1:
        for j in g(w[i+2]):
             m[k[w[i+1]+j]]=v[w[i]+j]
    else:
        for j in g(l(w[i+1])):
            m[k[w[i+1][j]]]=v[w[i]+j]
a.dump(m,open(o,'w'))""".format(str(sporadic_seg))

    document2 = """import json as a,sys
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
a.dump(dict(zip(s(k),s(v))),open(o,'w'))"""

    #script=open(script_name,'w')
    #script.write(document1 if len(sporadic_seg) != 0 else document2)
    #script.close()

same: 448
prefix: 225
infix: 0
postfix: 0
others: 10643
('INCROSS_LAG_VIPS/TIU_FIGS_owk1_sai[0]_DR_owk1_sai[1]_DR_owk1_sai[2]_DR_owk1_sai[3]/unta_$O1/lao1/O$1', 'INCROSS_LAG_VIPS/owk1_sai[0]/O$1', ['1', '0', '1', '1', '1', '2', '1', '3', '1', '1', '1'], 'INCROSS_LAG_VIPS/TIU_FIGS_owk_sai[]_DR_owk_sai[]_DR_owk_sai[]_DR_owk_sai[]/unta_$O/lao/O$')
('INCROSS_LAG_VIPS/TIU_FIGS_owk1_sai[0]_DR_owk1_sai[1]_DR_owk1_sai[2]_DR_owk1_sai[3]/unta_$O2/lao1/O$1', 'INCROSS_LAG_VIPS/owk1_sai[1]/O$1', ['1', '0', '1', '1', '1', '2', '1', '3', '2', '1', '1'], 'INCROSS_LAG_VIPS/TIU_FIGS_owk_sai[]_DR_owk_sai[]_DR_owk_sai[]_DR_owk_sai[]/unta_$O/lao/O$')
('INCROSS_LAG_VIPS/TIU_FIGS_owk1_sai[0]_DR_owk1_sai[1]_DR_owk1_sai[2]_DR_owk1_sai[3]/unta_$O3/lao1/O$1', 'INCROSS_LAG_VIPS/owk1_sai[2]/O$1', ['1', '0', '1', '1', '1', '2', '1', '3', '3', '1', '1'], 'INCROSS_LAG_VIPS/TIU_FIGS_owk_sai[]_DR_owk_sai[]_DR_owk_sai[]_DR_owk_sai[]/unta_$O/lao/O$')
('INCROSS_LAG_VIPS/TIU_FIGS_owk1_sai[0]_DR_owk1_sai[1]_DR_owk1_sai[2]_D

TypeError: Function takes at least 1 positional arguments (0 given)

In [145]:
sss = 'mawp_glt_dhoti_beg_ket[23]'
tt = '23'
sss.replace(tt,'')

'mawp_glt_dhoti_beg_ket[]'